# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [100]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [101]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [102]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [103]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [104]:
# Step 3: Read the CSV file
df = pd.read_csv('event_datafile_new.csv')

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [105]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [106]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [107]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# 1A. Create table and insert data in table for query 1

In [108]:
query = "CREATE TABLE IF NOT EXISTS music_app_session"
query = query + " (session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [109]:
query = "INSERT INTO music_app_session (session_id, item_in_session, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"


# Insert each row into table
for index, row in df.iterrows():
    session.execute(query, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))


# 1B. Do a SELECT to verify that the data have been inserted into music_app_session

In [110]:
query = "SELECT artist, song, length FROM music_app_session"
try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

# 1C. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [111]:
# The below query provides the artist, song, and length when the session_id is 338 and item_in_session is 4. The primary key for the table is session_id and item_in_session since those are both used in the where clause

In [112]:

query = "SELECT artist, song, length FROM music_app_session WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length,)
                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# 2A. Create table and insert data in table for query 2

In [113]:
query = "CREATE TABLE IF NOT EXISTS music_app_user_session "
query = query + "(session_id int, user_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY (( session_id, user_id), item_in_session ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [114]:
query = "INSERT INTO music_app_user_session (session_id, user_id, item_in_session, artist, song, first_name, last_name)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"


# Insert each row into table
for index, row in df.iterrows():
    session.execute(query, (row['sessionId'], row['userId'], row['itemInSession'], row['artist'], row['song'], row['firstName'], 
                                          row['lastName']
                                            ))


# 2B. Do a SELECT to verify that the data have been inserted into music_app_user_session

In [115]:
query = "SELECT item_in_session, artist, song, first_name, last_name FROM music_app_user_session"
try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

# 2C. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

# The below query provides the artist, song, users first and last name when the user_id is 10 and session_id is 182. The data is sorted by item_in_session. The primary key for this data is session_id and user_id since those are in the where clause and it is clustered by item_in_session.

In [116]:

query = "SELECT item_in_session, artist, song, first_name, last_name FROM music_app_user_session WHERE  session_id=182 and user_id=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.item_in_session, row.first_name, row.last_name)
                    
                    

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


# 3A. Create table and insert data in table for query 3

In [134]:
query = "CREATE TABLE IF NOT EXISTS music_app_songs "
query = query + "(  song text, user_id int, first_name text, last_name text,  PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [135]:
query = "INSERT INTO music_app_songs ( song, user_id, first_name, last_name)"
query = query + " VALUES (%s, %s, %s, %s)"


# Insert each row into table
for index, row in df.iterrows():
    session.execute(query, (  row['song'], row['userId'], row['firstName'], row['lastName']))


# 3B. Do a SELECT to verify that the data have been inserted into music_app_songs

In [136]:
query = "SELECT song, first_name, last_name FROM music_app_user_session"
try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

# 3C. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# The query below selects the first and last name of anyone who listened to the song All Hands Against His Own. The table has a primary key of song since it is used in the where clause.

In [137]:
query = " SELECT DISTINCT user_id ,first_name, last_name FROM music_app_user_session WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.first_name, row.last_name)
                    

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Jacob Klein
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Koch
Lily Ko

### Drop all tables before closing out the sessions

In [99]:
for table in ["music_app_session", "music_app_user_session", "music_app_songs"]:
    query = f"DROP TABLE {table}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


### Close the session and cluster connection¶

In [98]:
session.shutdown()
cluster.shutdown()